In [1]:
from bayes_opt import BayesianOptimization

In [ ]:
train_x = df_train[feat]
train_y = df_train['loanstatus']

In [ ]:
xgtrain = xgb.DMatrix(train_x, label = train_y, missing = np.NAN)

In [ ]:
def xgb_evaluate(min_xhild_weight,
                colsample_bytree,
                max_depth,
                subsample,
                gamma):
    params = dict()
    params['objective'] = 'binary:logistic'
    params['eta'] = 0.05
    params['max_depth'] = int(max_depth)
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = solsample_bytree
    params['subsample'] = subsample
    params['gamma'] = gamma
    params['verbose_eval'] = False
    
    cv_result = xgb.cv(params, xgtrain,
                      num_boost_round = 100000,
                      nfold = 3,
                      metrics = {'auc'},
                      seed = 1234,
                      callback = [xgb.callback.early_stop[50]])
    print(cv_result)
    
    return cv_result['test-auc-mean'].max()


In [ ]:
xgb_BO = BayesianOptimization(xgb_evaluate,
                             {'max_depth':(4, 8),
                             'min_child_weight':(0, 20),
                             'colsample_bytree':(0.2, 0.8),
                             'subsample':(0.5, 1),
                             'gamma':(0,2)
                             }
                             )
xgb_BO.maximize(init_points = 5, n_iter = 40)